# Sample data preprocessing script

This script prepares the sample data used for debugging. The data is the EEG data from the [Kaggle](https://www.kaggle.com/datasets/wanghaohan/confused-eeg) repository. 

**References:**
Wang, H., Li, Y., Hu, X., Yang, Y., Meng, Z., & Chang, K. M. (2013, June). Using EEG to Improve Massive Open Online Courses Feedback Interaction. In AIED Workshops.

In [122]:
import pandas as pd
import numpy as np

# Specify the path to the dataset (i.e. the downloaded Kaggle dataset) and the path where to save the processed dataset
DATA_PATH = '~/Documents/Thesis/sample_data/EEG_data.csv'
SAVE_PATH = '~/Git/thesis-repo/data/sample/data.csv'
N_SEGMENTS = 4
NUMERIC_FEATURES = ['Attention', 'Mediation', 'Raw', 'Delta', 'Theta', 'Alpha1', 'Alpha2', 'Beta1', 'Beta2', 'Gamma1', 'Gamma2']

In [123]:
# Load the dataset from the specified path
df = pd.read_csv(DATA_PATH)
# Show the sample of the dataset
df.head()

,SubjectID,VideoID,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel,user-definedlabeln
0,0.0,0.0,56.0,43.0,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0,0.0
1,0.0,0.0,40.0,35.0,-50.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0.0,0.0
2,0.0,0.0,47.0,48.0,101.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,0.0,0.0
3,0.0,0.0,47.0,57.0,-5.0,2012240.0,129350.0,61236.0,17084.0,11488.0,62462.0,49960.0,33932.0,0.0,0.0
4,0.0,0.0,44.0,53.0,-8.0,1005145.0,354328.0,37102.0,88881.0,45307.0,99603.0,44790.0,29749.0,0.0,0.0


In [124]:
# Store the new dataset in the csv file
df.drop(columns = ['predefinedlabel']).to_csv(SAVE_PATH)